In [61]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")

### Dataset 

In [62]:
df = pd.read_csv(r"D:\PYTON PROGRAMMING\PYTHON FILES\Scikit-Learn\Feature Engineering\laptops.csv")

In [63]:
df.head(5)

,Laptop,Status,Brand,Model,CPU,RAM,Storage,Storage type,GPU,Screen,Touch,Final Price
0,ASUS ExpertBook B1 B1502CBA-EJ0436X Intel Core...,New,Asus,ExpertBook,Intel Core i5,8,512,SSD,NaN,15.6,No,1009.00
1,Alurin Go Start Intel Celeron N4020/8GB/256GB ...,New,Alurin,Go,Intel Celeron,8,256,SSD,NaN,15.6,No,299.00
2,ASUS ExpertBook B1 B1502CBA-EJ0424X Intel Core...,New,Asus,ExpertBook,Intel Core i3,8,256,SSD,NaN,15.6,No,789.00
3,MSI Katana GF66 12UC-082XES Intel Core i7-1270...,New,MSI,Katana,Intel Core i7,16,1000,SSD,RTX 3050,15.6,No,1199.00
4,HP 15S-FQ5085NS Intel Core i5-1235U/16GB/512GB...,New,HP,15S,Intel Core i5,16,512,SSD,NaN,15.6,No,669.01


In [68]:
from sklearn.model_selection import train_test_split

X = df[['RAM', 'Storage']].to_numpy().reshape(-1, 2)
Y = df['Final Price']

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

from sklearn.linear_model import LinearRegression

model = LinearRegression()
model.fit(X_train, Y_train)

Y_pred = model.predict(X_test)

from sklearn.metrics import r2_score

r2 = r2_score(Y_test, Y_pred)
print(f"R-squared: {r2}")

from sklearn.linear_model import Lasso

lasso = Lasso()
lasso.fit(X_train, Y_train)

Y_pred = lasso.predict(X_test)

r2 = r2_score(Y_test, Y_pred)
print(f"R-squared: {r2}")

R-squared: 0.574430192515315
R-squared: 0.5744857247405207


In [47]:
df[(df['Brand'] == 'Lenovo') & (df['CPU'].str.contains('Intel'))]['Laptop']

10      Lenovo V15 IGL Intel Celeron N4020/8GB/256 GB ...
16      Lenovo IdeaPad 3 15IAU7 Intel Core i5-1235U/16...
29      Lenovo IdeaPad 3 15ITL6 Intel Core i5-1155G7/8...
46      Lenovo IdeaPad Gaming 3 15IHU6 Intel Core i5-1...
51      Lenovo ThinkBook 14 Intel Core i5-1135G7/8GB/2...
                              ...                        
1994    Lenovo V330-15IKB Intel Core i3-8130U/4GB/128G...
1995    Lenovo Yoga 9 14ITL5 Intel Evo Core i7-1185G7/...
1996    Lenovo Yoga C930-13IKB Intel Core i5-8250U/8GB...
1997    Lenovo Yoga Creator 7 15IMH05 Intel Core i7-10...
1998    Lenovo Yoga S940-14IIL Intel Core i7-1065G7/16...
Name: Laptop, Length: 260, dtype: object

In [84]:
def asus_intel_core_extract_release_year(laptop_name):      # For Asus Intel Core
    import re

    # Search for the processor pattern like i5-1235U
    match = re.search(r'i[3579]-\d{4,5}[A-Z]?', laptop_name, re.IGNORECASE)
    
    if match:
        cpu_model = match.group()
        gen_digits = re.search(r'\d{4,5}', cpu_model)
        
        if gen_digits:
            gen = gen_digits.group()
            # For 10th gen and above
            generation = int(gen[:2]) if gen.startswith('1') else int(gen[0])
            
            # Map generation to approximate release year
            release_year = 2010 + generation  # 1st gen → 2011, 12th gen → 2022
            return release_year
    return "Unknown"
def asus_extract_ryzen_release_year(laptop_name):
    import re

    # Find Ryzen processor pattern, like Ryzen 5 5500U or Ryzen 7 5800H
    match = re.search(r'Ryzen\s+\d\s+(\d{4,5})[A-Z]?', laptop_name, re.IGNORECASE)
    
    if match:
        model_number = match.group(1)  # e.g., 5800 from 5800H

        try:
            series = int(model_number[:4])  # Use first 4 digits (some models are 5 digits)
        except:
            return "Unknown"

        # Map Ryzen series to release years
        if 1000 <= series < 2000:
            return 2017
        elif 2000 <= series < 3000:
            return 2018
        elif 3000 <= series < 4000:
            return 2019  # mostly desktops
        elif 4000 <= series < 5000:
            return 2020
        elif 5000 <= series < 6000:
            return 2021
        elif 6000 <= series < 7000:
            return 2022
        elif 7000 <= series < 8000:
            return 2023
        elif 8000 <= series < 9000:
            return 2024
        else:
            return "Unknown"
    else:
        return "Unknown"
def msi_extract_intel_release_year(laptop_name):
    import re

    # Match Intel CPU pattern like i5-1135G7, i7-11800H, etc.
    match = re.search(r'i[3579]-\d{4,5}[A-Z]?', laptop_name, re.IGNORECASE)

    if match:
        cpu_model = match.group()
        digits_match = re.search(r'\d{4,5}', cpu_model)

        if digits_match:
            digits = digits_match.group()

            if digits.startswith('10'):
                generation = 10
            elif digits.startswith('11'):
                generation = 11
            elif digits.startswith('12'):
                generation = 12
            elif digits.startswith('13'):
                generation = 13
            elif digits.startswith('14'):
                generation = 14
            else:
                generation = int(digits[0])  # For older gen (i7-7700HQ → 7th gen)

            # Intel 1st gen = 2010, so offset
            return 2009 + generation

    return "Unknown"
def msi_extract_ryzen_release_year(laptop_name):
    import re

    # Match Ryzen CPU pattern like Ryzen 5 7530U, Ryzen 7 5800H, etc.
    match = re.search(r'Ryzen\s+\d\s+\d{4,5}[A-Z]?', laptop_name, re.IGNORECASE)

    if match:
        cpu_model = match.group()
        digits_match = re.search(r'\d{4,5}', cpu_model)

        if digits_match:
            digits = digits_match.group()
            series_digit = int(digits[0])  # e.g., '7' from '7530'

            # Series-to-year mapping (approximate, based on AMD naming conventions)
            series_year_map = {
                7: 2023,
                6: 2022,
                5: 2021,
                4: 2020,
                3: 2019,
                2: 2018,
                1: 2017
            }

            return series_year_map.get(series_digit, "Unknown")

    return "Unknown"
def hp_extract_ryzen_release_year(laptop_name):
    import re

    # Match Ryzen CPU pattern like Ryzen 5 5500U, Ryzen 7 5800H, etc.
    match = re.search(r'Ryzen\s+\d\s+\d{4,5}[A-Z]?', laptop_name, re.IGNORECASE)

    if match:
        cpu_model = match.group()
        digits_match = re.search(r'\d{4,5}', cpu_model)

        if digits_match:
            digits = digits_match.group()
            series_digit = int(digits[0])  # e.g., '5' from '5500U'

            # Ryzen series generation to release year mapping (approximate)
            series_year_map = {
                7: 2023,
                6: 2022,
                5: 2021,
                4: 2020,
                3: 2019,
                2: 2018,
                1: 2017
            }

            return series_year_map.get(series_digit, "Unknown")

    return "Unknown"
def hp_extract_intel_release_year(laptop_name):
    import re

    # Look for Intel processor pattern like i5-1235U, i7-11800H, etc.
    match = re.search(r'i[3579]-\d{4,5}[A-Z]?', laptop_name, re.IGNORECASE)

    if match:
        cpu_model = match.group()
        gen_digits = re.search(r'\d{4,5}', cpu_model)

        if gen_digits:
            digits = gen_digits.group()
            # For 10th gen and above: first 2 digits = generation
            generation = int(digits[:2]) if digits.startswith('1') else int(digits[0])

            return 2010 + generation  # 1st gen → 2011, 12th gen → 2022

    return "Unknown"
def lenovo_extract_intel_release_year_universal(laptop_name):
    import re

    # Handle i3/i5/i7/i9 processors
    match_i_series = re.search(r'i[3579]-\d{4,5}[A-Z]?', laptop_name, re.IGNORECASE)
    if match_i_series:
        cpu_model = match_i_series.group()
        digits = re.search(r'\d{4,5}', cpu_model)
        if digits:
            gen = digits.group()
            generation = int(gen[:2]) if gen.startswith('1') else int(gen[0])
            return 2010 + generation  # 1st gen = 2011, 12th gen = 2022

    # Handle Intel Celeron
    match_celeron = re.search(r'Celeron\s+N\d{4}', laptop_name, re.IGNORECASE)
    if match_celeron:
        model = re.search(r'N\d{4}', match_celeron.group())
        if model:
            model_num = int(model.group()[1:])
            # Approx mapping based on release data
            if model_num >= 5000:
                return 2023
            elif model_num >= 4500:
                return 2022
            elif model_num >= 4100:
                return 2020
            elif model_num >= 4000:
                return 2019
            else:
                return 2018

    # Handle Intel Pentium (optional extension)
    match_pentium = re.search(r'Pentium\s+[A-Z]?\d{4,5}', laptop_name, re.IGNORECASE)
    if match_pentium:
        return "Est. between 2018–2021"

    return "Unknown"
def extract_lenovo_ryzen_year(laptop_name):
    import re
    
    # Check for AMD Ryzen CPU
    match = re.search(r'Ryzen\s+[3579]\s+(\d{4})[A-Z]?', laptop_name, re.IGNORECASE)
    if match:
        model_number = match.group(1)
        if len(model_number) == 4:
            generation = int(model_number[0])
            return 2016 + generation  # AMD Ryzen 1st gen was in 2017
    return "Unknown"
def extract_acer_release_year(laptop_name):
    import re

    # First check for Intel processors
    intel_match = re.search(r'i[3579]-\d{4,5}[A-Z]?', laptop_name, re.IGNORECASE)
    if intel_match:
        cpu_model = intel_match.group()
        digits = re.search(r'\d{4,5}', cpu_model)
        if digits:
            gen = digits.group()
            generation = int(gen[:2]) if gen.startswith('1') else int(gen[0])
            return 2010 + generation  # i3/5/7 1st gen → 2011, 12th → 2022

    # Then check for AMD Ryzen processors
    amd_match = re.search(r'Ryzen\s+[3579]\s+(\d{4})[A-Z]?', laptop_name, re.IGNORECASE)
    if amd_match:
        model_number = amd_match.group(1)
        if len(model_number) == 4:
            generation = int(model_number[0])
            return 2016 + generation  # Ryzen 1000 series → 2017

    return "Unknown"
def extract_apple_release_year(laptop_name):
    import re

    m_series_match = re.search(r'\bM([123])\b', laptop_name, re.IGNORECASE)
    if m_series_match:
        m_chip = int(m_series_match.group(1))
        return 2019 + m_chip  # M1 → 2020, M2 → 2021, M3 → 2022

    return "Unknown"
def extract_dell_release_year(laptop_name):
    import re

    # Check for Intel Core pattern: i3, i5, i7, i9
    intel_match = re.search(r'i[3579]-\d{4,5}[A-Z]?', laptop_name, re.IGNORECASE)
    if intel_match:
        cpu_model = intel_match.group()
        digits = re.search(r'\d{4,5}', cpu_model)
        if digits:
            gen = digits.group()
            generation = int(gen[:2]) if gen.startswith('1') else int(gen[0])
            return 2010 + generation  # i3-10110U → 10th gen → 2020

    # Check for AMD Ryzen pattern: Ryzen 5 5500U, Ryzen 7 5800H, etc.
    amd_match = re.search(r'Ryzen\s*[3579]\s*(\d{4})', laptop_name, re.IGNORECASE)
    if amd_match:
        model_num = int(amd_match.group(1))
        series = model_num // 100  # 5500U → 55 → series 5
        # Rough mapping (AMD release is trickier than Intel)
        if series == 3:
            return 2019
        elif series == 4:
            return 2020
        elif series == 5:
            return 2021
        elif series == 6:
            return 2022
        elif series == 7:
            return 2023
        elif series == 8:
            return 2024

    return "Unknown"
def extract_microsoft_release_year(laptop_name):
    import re

    # INTEL CPU extraction (e.g., i5-1135G7, i7-1255U)
    intel_match = re.search(r'i[3579]-\d{4,5}[A-Z]?', laptop_name, re.IGNORECASE)
    if intel_match:
        digits = re.search(r'\d{4,5}', intel_match.group())
        if digits:
            gen = digits.group()
            generation = int(gen[:2]) if gen.startswith('1') else int(gen[0])
            return 2010 + generation  # i5-1135G7 → 11th Gen → 2021

    # AMD Ryzen CPU extraction (e.g., Ryzen 5 5500U, Ryzen 7 7730U)
    amd_match = re.search(r'Ryzen\s*[3579]\s*(\d{4})', laptop_name, re.IGNORECASE)
    if amd_match:
        model = int(amd_match.group(1))
        series = model // 100  # 5500 → 55
        # Approximate mapping
        if series == 3:
            return 2019
        elif series == 4:
            return 2020
        elif series == 5:
            return 2021
        elif series == 6:
            return 2022
        elif series == 7:
            return 2023
        elif series == 8:
            return 2024

    return "Unknown"
def extract_razer_release_year(laptop_name):
    import re

    # Check for Intel processor
    intel_match = re.search(r'i[3579]-\d{4,5}[A-Z]?', laptop_name, re.IGNORECASE)
    if intel_match:
        digits = re.search(r'\d{4,5}', intel_match.group())
        if digits:
            gen = digits.group()
            generation = int(gen[:2]) if gen.startswith('1') else int(gen[0])
            return 2010 + generation  # i5-12500H → 12th gen → 2022

    # Check for AMD Ryzen processor
    amd_match = re.search(r'Ryzen\s*[3579]\s*(\d{4})', laptop_name, re.IGNORECASE)
    if amd_match:
        model = int(amd_match.group(1))
        series = model // 100  # 5800 → 58

        # Approximate year mapping
        if series == 3:
            return 2019
        elif series == 4:
            return 2020
        elif series == 5:
            return 2021
        elif series == 6:
            return 2022
        elif series == 7:
            return 2023
        elif series == 8:
            return 2024

    return "Unknown"
def extract_medion_release_year(laptop_name):
    import re

    # Intel Processor Check
    intel_match = re.search(r'i[3579]-\d{4,5}[A-Z]?', laptop_name, re.IGNORECASE)
    if intel_match:
        digits = re.search(r'\d{4,5}', intel_match.group())
        if digits:
            gen = digits.group()
            generation = int(gen[:2]) if gen.startswith('1') else int(gen[0])
            return 2010 + generation  # e.g., i5-1235U → 12th gen → 2022

    # AMD Ryzen Processor Check
    amd_match = re.search(r'Ryzen\s*[3579]\s*(\d{4})', laptop_name, re.IGNORECASE)
    if amd_match:
        model = int(amd_match.group(1))
        series = model // 100  # 5700U → 57

        # Estimate release year based on Ryzen series
        if series == 3:
            return 2019
        elif series == 4:
            return 2020
        elif series == 5:
            return 2021
        elif series == 6:
            return 2022
        elif series == 7:
            return 2023
        elif series == 8:
            return 2024

    return "Unknown"
def extract_lg_release_year(laptop_name):
    import re

    # Intel Core processors
    intel_match = re.search(r'i[3579]-\d{4,5}[A-Z]?', laptop_name, re.IGNORECASE)
    if intel_match:
        digits = re.search(r'\d{4,5}', intel_match.group())
        if digits:
            number = digits.group()
            # Logic: if it starts with 12 → 12th gen → 2022
            generation = int(number[:2]) if number.startswith('1') else int(number[0])
            return 2010 + generation

    # AMD Ryzen processors
    amd_match = re.search(r'Ryzen\s*[3579]\s*(\d{4})', laptop_name, re.IGNORECASE)
    if amd_match:
        model_num = int(amd_match.group(1))
        series = model_num // 100

        # Rough mapping by Ryzen series
        if series == 3:
            return 2019
        elif series == 4:
            return 2020
        elif series == 5:
            return 2021
        elif series == 6:
            return 2022
        elif series == 7:
            return 2023
        elif series == 8:
            return 2024

    return "Unknown"
def extract_alurin_release_year(laptop_name):
    import re

    # Intel Core CPUs
    intel_match = re.search(r'i[3579]-\d{4,5}[A-Z]?', laptop_name, re.IGNORECASE)
    if intel_match:
        digits = re.search(r'\d{4,5}', intel_match.group())
        if digits:
            number = digits.group()
            gen = int(number[:2]) if number.startswith('12') or number.startswith('11') else int(number[0])
            return 2010 + gen

    # AMD Ryzen CPUs
    amd_match = re.search(r'Ryzen\s*[3579]\s*(\d{4})', laptop_name, re.IGNORECASE)
    if amd_match:
        model_num = int(amd_match.group(1))
        series = model_num // 100
        ryzen_year_map = {
            3: 2019,
            4: 2020,
            5: 2021,
            6: 2022,
            7: 2023,
            8: 2024
        }
        return ryzen_year_map.get(series, "Unknown")

    return "Unknown"
def extract_pccom_release_year(laptop_name):
    import re

    # Intel Core CPUs
    intel_match = re.search(r'i[3579]-\d{4,5}[A-Z]?', laptop_name, re.IGNORECASE)
    if intel_match:
        digits = re.search(r'\d{4,5}', intel_match.group())
        if digits:
            number = digits.group()
            gen = int(number[:2]) if number.startswith('12') or number.startswith('11') else int(number[0])
            return 2010 + gen

    # AMD Ryzen CPUs
    amd_match = re.search(r'Ryzen\s*[3579]\s*(\d{4})', laptop_name, re.IGNORECASE)
    if amd_match:
        model_num = int(amd_match.group(1))
        series = model_num // 100
        ryzen_year_map = {
            3: 2019,
            4: 2020,
            5: 2021,
            6: 2022,
            7: 2023,
            8: 2024
        }
        return ryzen_year_map.get(series, "Unknown")

    return "Unknown"
def extract_samsung_release_year(laptop_name):
    import re

    # Intel CPUs
    intel_match = re.search(r'i[3579]-\d{4,5}[A-Z]?', laptop_name, re.IGNORECASE)
    if intel_match:
        digits = re.search(r'\d{4,5}', intel_match.group())
        if digits:
            number = digits.group()
            # 11th Gen and later
            if number.startswith("12") or number.startswith("13") or number.startswith("14"):
                gen = int(number[:2])
            else:  # Older gens
                gen = int(number[0])
            return 2010 + gen

    # AMD Ryzen CPUs
    amd_match = re.search(r'Ryzen\s*[3579]\s*(\d{4})', laptop_name, re.IGNORECASE)
    if amd_match:
        model_num = int(amd_match.group(1))
        series = model_num // 100
        ryzen_year_map = {
            3: 2019,
            4: 2020,
            5: 2021,
            6: 2022,
            7: 2023,
            8: 2024
        }
        return ryzen_year_map.get(series, "Unknown")

    return "Unknown"
def extract_dynabook_release_year(laptop_name):
    import re

    # Intel CPU pattern
    intel_match = re.search(r'i[3579]-\d{4,5}[A-Z]?', laptop_name, re.IGNORECASE)
    if intel_match:
        digits = re.search(r'\d{4,5}', intel_match.group())
        if digits:
            number = digits.group()
            # For 12th gen and beyond
            if number.startswith("12") or number.startswith("13") or number.startswith("14"):
                gen = int(number[:2])
            else:
                gen = int(number[0])
            return 2010 + gen

    # AMD Ryzen pattern
    amd_match = re.search(r'Ryzen\s*[3579]\s*(\d{4})', laptop_name, re.IGNORECASE)
    if amd_match:
        model_num = int(amd_match.group(1))
        series = model_num // 100
        ryzen_year_map = {
            3: 2019,
            4: 2020,
            5: 2021,
            6: 2022,
            7: 2023,
            8: 2024
        }
        return ryzen_year_map.get(series, "Unknown")

    return "Unknown"
def extract_primux_release_year(laptop_name):
    import re

    # Match Intel Core processor (e.g., i5-1235U, i3-10110U)
    intel_match = re.search(r'i[3579]-\d{4,5}[A-Z]?', laptop_name, re.IGNORECASE)
    if intel_match:
        digits = re.search(r'\d{4,5}', intel_match.group())
        if digits:
            number = digits.group()
            # Handle 12th, 13th, 14th gen etc.
            if number.startswith(("12", "13", "14")):
                gen = int(number[:2])
            else:
                gen = int(number[0])
            return 2010 + gen

    # Match AMD Ryzen processor (e.g., Ryzen 5 5500U)
    amd_match = re.search(r'Ryzen\s*[3579]\s*(\d{4})', laptop_name, re.IGNORECASE)
    if amd_match:
        model_number = int(amd_match.group(1))
        series = model_number // 100
        ryzen_year_map = {
            3: 2019,
            4: 2020,
            5: 2021,
            6: 2022,
            7: 2023,
            8: 2024
        }
        return ryzen_year_map.get(series, "Unknown")

    return "Unknown"
def extract_innjoo_release_year(laptop_name):
    import re

    # Match Intel Core processor (e.g., i3-10110U, i5-1235U, i7-8550U)
    intel_match = re.search(r'i[3579]-\d{4,5}[A-Z]?', laptop_name, re.IGNORECASE)
    if intel_match:
        digits = re.search(r'\d{4,5}', intel_match.group())
        if digits:
            number = digits.group()
            if number.startswith(("12", "13", "14")):
                gen = int(number[:2])  # e.g., 12th Gen
            else:
                gen = int(number[0])   # e.g., 8th Gen = '8'
            return 2010 + gen

    # Match AMD Ryzen processor (e.g., Ryzen 5 5500U)
    amd_match = re.search(r'Ryzen\s*[3579]\s*(\d{4})', laptop_name, re.IGNORECASE)
    if amd_match:
        model_number = int(amd_match.group(1))
        series = model_number // 100  # e.g., 5500 -> 5th Gen
        ryzen_year_map = {
            3: 2019,
            4: 2020,
            5: 2021,
            6: 2022,
            7: 2023,
            8: 2024
        }
        return ryzen_year_map.get(series, "Unknown")

    return "Unknown"
def extract_prixton_release_year(laptop_name):
    import re

    # Intel Core series extraction
    intel_match = re.search(r'i[3579]-\d{4,5}[A-Z]?', laptop_name, re.IGNORECASE)
    if intel_match:
        digits = re.search(r'\d{4,5}', intel_match.group())
        if digits:
            num = digits.group()
            if num.startswith(('12', '13', '14')):  # Modern generations (12th+)
                generation = int(num[:2])
            else:  # Older generations (e.g., i5-8250U -> 8th gen)
                generation = int(num[0])
            return 2010 + generation

    # AMD Ryzen series extraction
    amd_match = re.search(r'Ryzen\s*[3579]\s*(\d{4})', laptop_name, re.IGNORECASE)
    if amd_match:
        model = int(amd_match.group(1))
        series = model // 100
        ryzen_year_map = {
            3: 2019,
            4: 2020,
            5: 2021,
            6: 2022,
            7: 2023,
            8: 2024
        }
        return ryzen_year_map.get(series, "Unknown")

    return "Unknown"
def extract_millenium_release_year(laptop_name):
    import re

    # Intel Core series
    intel_match = re.search(r'i[3579]-\d{4,5}[A-Z]?', laptop_name, re.IGNORECASE)
    if intel_match:
        digits = re.search(r'\d{4,5}', intel_match.group())
        if digits:
            num = digits.group()
            if num.startswith(('12', '13', '14')):  # Modern generations (12th+)
                generation = int(num[:2])
            else:  # Older generations (e.g., i7-8750H = 8th gen)
                generation = int(num[0])
            return 2010 + generation

    # AMD Ryzen series
    amd_match = re.search(r'Ryzen\s*[3579]\s*(\d{4})', laptop_name, re.IGNORECASE)
    if amd_match:
        model = int(amd_match.group(1))
        series = model // 100
        ryzen_year_map = {
            3: 2019,
            4: 2020,
            5: 2021,
            6: 2022,
            7: 2023,
            8: 2024
        }
        return ryzen_year_map.get(series, "Unknown")

    return "Unknown"
def extract_denver_release_year(laptop_name):
    import re

    # Check for Intel CPUs
    intel_match = re.search(r'i[3579]-\d{4,5}[A-Z]?', laptop_name, re.IGNORECASE)
    if intel_match:
        digits = re.search(r'\d{4,5}', intel_match.group())
        if digits:
            num = digits.group()
            if num.startswith(('12', '13', '14')):
                generation = int(num[:2])
            else:
                generation = int(num[0])
            return 2010 + generation

    # Check for AMD Ryzen CPUs
    amd_match = re.search(r'Ryzen\s*[3579]\s*(\d{4})', laptop_name, re.IGNORECASE)
    if amd_match:
        model_number = int(amd_match.group(1))
        series = model_number // 100
        ryzen_map = {
            3: 2019,
            4: 2020,
            5: 2021,
            6: 2022,
            7: 2023,
            8: 2024
        }
        return ryzen_map.get(series, "Unknown")

    return "Unknown"
def extract_jetwing_release_year(laptop_name):
    import re

    # INTEL processor match
    intel_match = re.search(r'i[3579]-\d{4,5}[A-Z]?', laptop_name, re.IGNORECASE)
    if intel_match:
        digits = re.search(r'\d{4,5}', intel_match.group())
        if digits:
            num = digits.group()
            if num.startswith(('12', '13', '14')):
                generation = int(num[:2])
            else:
                generation = int(num[0])
            return 2010 + generation

    # AMD Ryzen match
    amd_match = re.search(r'Ryzen\s*[3579]\s*(\d{4})', laptop_name, re.IGNORECASE)
    if amd_match:
        model_number = int(amd_match.group(1))
        series = model_number // 100
        ryzen_map = {
            3: 2019,
            4: 2020,
            5: 2021,
            6: 2022,
            7: 2023,
            8: 2024
        }
        return ryzen_map.get(series, "Unknown")

    return "Unknown"
def extract_realme_release_year(laptop_name):
    import re

    # INTEL processor match
    intel_match = re.search(r'i[3579]-\d{4,5}[A-Z]?', laptop_name, re.IGNORECASE)
    if intel_match:
        digits = re.search(r'\d{4,5}', intel_match.group())
        if digits:
            num = digits.group()
            if num.startswith(('12', '13', '14')):
                generation = int(num[:2])
            else:
                generation = int(num[0])
            return 2010 + generation

    # AMD Ryzen match
    amd_match = re.search(r'Ryzen\s*[3579]\s*(\d{4})', laptop_name, re.IGNORECASE)
    if amd_match:
        model_number = int(amd_match.group(1))
        series = model_number // 100
        ryzen_map = {
            3: 2019,
            4: 2020,
            5: 2021,
            6: 2022,
            7: 2023,
            8: 2024
        }
        return ryzen_map.get(series, "Unknown")

    return "Unknown"


In [92]:
# Apply function for each condition
df.loc[(df['Brand'] == 'Asus') & df['CPU'].str.contains('Intel', na=False), 'release_year'] = df['Laptop'].apply(asus_intel_core_extract_release_year)
df.loc[(df['Brand'] == 'Asus') & df['CPU'].str.contains('Ryzen', na=False), 'release_year'] = df['Laptop'].apply(asus_extract_ryzen_release_year)

df.loc[(df['Brand'] == 'MSI') & df['CPU'].str.contains('Intel', na=False), 'release_year'] = df['Laptop'].apply(msi_extract_intel_release_year)
df.loc[(df['Brand'] == 'MSI') & df['CPU'].str.contains('Ryzen', na=False), 'release_year'] = df['Laptop'].apply(msi_extract_ryzen_release_year)

df.loc[(df['Brand'] == 'HP') & df['CPU'].str.contains('Intel', na=False), 'release_year'] = df['Laptop'].apply(hp_extract_intel_release_year)
df.loc[(df['Brand'] == 'HP') & df['CPU'].str.contains('Ryzen', na=False), 'release_year'] = df['Laptop'].apply(hp_extract_ryzen_release_year)

df.loc[(df['Brand'] == 'Lenovo') & df['CPU'].str.contains('Intel', na=False), 'release_year'] = df['Laptop'].apply(lenovo_extract_intel_release_year_universal)
df.loc[(df['Brand'] == 'Lenovo') & df['CPU'].str.contains('Ryzen', na=False), 'release_year'] = df['Laptop'].apply(extract_lenovo_ryzen_year)

df.loc[df['Brand'] == 'Acer', 'release_year'] = df['Laptop'].apply(extract_acer_release_year)
df.loc[df['Brand'] == 'Dell', 'release_year'] = df['Laptop'].apply(extract_dell_release_year)
df.loc[df['Brand'] == 'Microsoft', 'release_year'] = df['Laptop'].apply(extract_microsoft_release_year)
df.loc[df['Brand'] == 'Apple', 'release_year'] = df['Laptop'].apply(extract_apple_release_year)
df.loc[df['Brand'] == 'Razer', 'release_year'] = df['Laptop'].apply(extract_razer_release_year)
df.loc[df['Brand'] == 'Medion', 'release_year'] = df['Laptop'].apply(extract_medion_release_year)
df.loc[df['Brand'] == 'LG', 'release_year'] = df['Laptop'].apply(extract_lg_release_year)
df.loc[df['Brand'] == 'Alurin', 'release_year'] = df['Laptop'].apply(extract_alurin_release_year)
df.loc[df['Brand'].str.upper() == 'PCCOM', 'release_year'] = df['Laptop'].apply(extract_pccom_release_year)
df.loc[df['Brand'] == 'Samsung', 'release_year'] = df['Laptop'].apply(extract_samsung_release_year)
df.loc[df['Brand'].str.contains('Dynabook|Toshiba', na=False), 'release_year'] = df['Laptop'].apply(extract_dynabook_release_year)
df.loc[df['Brand'] == 'Realme', 'release_year'] = df['Laptop'].apply(extract_realme_release_year)


In [95]:
df['release_year'].unique()

array([2022, 'Unknown', 2021, 2020, 2019, 2023, nan, 2011, 2016, 2017,
       2018, 2014, 2015], dtype=object)

In [96]:
df.to_csv(r"D:\PYTON PROGRAMMING\PYTHON FILES\Scikit-Learn\Feature Engineering\laptops_new.csv", index=False)

### Practice Standardization

In [23]:
df_movie.head()

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description,duration_int
0,s1,Movie,Dick Johnson Is Dead,Kirsten Johnson,NaN,United States,"September 25, 2021",2021,PG-13,90 min,Documentaries,"As her father nears the end of his life, filmm...",90.0
6,s7,Movie,My Little Pony: A New Generation,"Robert Cullen, José Luis Ucha","Vanessa Hudgens, Kimiko Glenn, James Marsden, ...",NaN,"September 24, 2021",2021,PG,91 min,Children & Family Movies,Equestria's divided. But a bright-eyed hero be...,91.0
7,s8,Movie,Sankofa,Haile Gerima,"Kofi Ghanaba, Oyafunmike Ogunlano, Alexandra D...","United States, Ghana, Burkina Faso, United Kin...","September 24, 2021",1993,TV-MA,125 min,"Dramas, Independent Movies, International Movies","On a photo shoot in Ghana, an American model s...",125.0
9,s10,Movie,The Starling,Theodore Melfi,"Melissa McCarthy, Chris O'Dowd, Kevin Kline, T...",United States,"September 24, 2021",2021,PG-13,104 min,"Comedies, Dramas",A woman adjusting to life after a loss contend...,104.0
12,s13,Movie,Je Suis Karl,Christian Schwochow,"Luna Wedler, Jannis Niewöhner, Milan Peschel, ...","Germany, Czech Republic","September 23, 2021",2021,TV-MA,127 min,"Dramas, International Movies",After most of her family is murdered in a terr...,127.0
